In [1]:
import torch
import numpy
ckpt = './experiment/train/v3/audio_3/model_epoch_50.pth'
from train import Network
from train.network.foundation_model import Network , AnglePredictor
foundation_model = AnglePredictor().to(torch.device('cuda'))
foundation_model.load_state_dict(torch.load(ckpt))
foundation_model.eval()

AnglePredictor(
  (audio_encoder): AudioEncoder(
    (conv2d_1): Conv2d(2, 128, kernel_size=(16, 16), stride=(16, 16), padding=(1, 1))
  )
  (fc1): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=128, bias=True)
    (2): ReLU()
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=8, bias=True)
  )
)

In [5]:
for name , param in foundation_model.named_parameters():
    print(name)

audio_encoder.conv2d_1.weight
audio_encoder.conv2d_1.bias
fc1.1.weight
fc1.1.bias
fc2.0.weight
fc2.0.bias
fc2.2.weight
fc2.2.bias


In [7]:
for name , param in foundation_model.named_parameters():
    if 'weight' in name and "fc2.0" in name:
        fc20 = param

In [23]:
import numpy as np
np.set_printoptions(suppress=True)

In [27]:
with open('param.txt' , 'w') as f:
    f.write(str(np.array(fc20.data.cpu())))

In [30]:
np.array(fc20.data.cpu())

array([[-0.00171588, -0.00270258,  0.00013805, ..., -0.00145565,
         0.05860132,  0.01285341],
       [-0.        ,  0.        , -0.        , ..., -0.        ,
         0.        ,  0.        ],
       [ 0.        , -0.        , -0.        , ..., -0.        ,
         0.        ,  0.        ],
       ...,
       [-0.00118748, -0.00073553,  0.00000102, ..., -0.00247664,
         0.00801164,  0.00915861],
       [ 0.        , -0.        ,  0.        , ..., -0.        ,
         0.        ,  0.        ],
       [-0.0006139 , -0.0036304 , -0.00004368, ...,  0.00229722,
        -0.00249339, -0.02061759]], dtype=float32)

In [ ]:
import numpy as np


In [11]:
import pickle
with open('val_loader.pkl' , 'rb') as f:
    data = pickle.load(f)

In [12]:
for batch in data:
    batch_visual, batch_audio, batch_action , batch_angle = batch
    break

In [13]:
batch_angle = batch_angle.float()

In [14]:
NUM_SECTORS = 8
SECTOR_ANGLE = 2 * 180 / NUM_SECTORS 
sector_label = torch.remainder(batch_angle + 180, 2*180) // SECTOR_ANGLE
sector_label = sector_label.long()

In [15]:
sector_label

tensor([3, 4, 4, 5, 2, 3, 4, 5, 0, 3, 3, 3, 4, 3, 4, 0, 7, 4, 1, 3, 2, 6, 3, 3,
        4, 5, 1, 4, 5, 3, 6, 0, 4, 2, 3, 0, 7, 2, 4, 3, 4, 3, 3, 3, 6, 5, 1, 4,
        2, 7, 5, 4, 6, 0, 4, 2, 2, 4, 5, 2, 4, 4, 4, 3])

In [16]:
torch.argmax(foundation_model(batch_audio.to('cuda')) , dim= 1)

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], device='cuda:0')